In [ ]:
#Import Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Introduction
Throughout history, one of the deadliest diseases humanity has faced is malaria. It is dubbed the "single biggest killer of humans in history" by the YouTube channel, 'Kurzgesagt - In a nutshell', who have a great video on its transmission, and ways scientists today are trying to fight it. One of the methods includes creating genetically modified mosquitoes, which are given immunity to malaria using CRISPR.

Per the CDC, malaria needs to be promptly treated to prevent further spread in the community, as well as prevent severe disease and potential death for the patient. But at the same time, they highly discourage "presumptive treatment" or treatment without confirmed case of malaria, as overuse of antimalarial drugs leads to drug-resistant strains, as well as unwanted side effects for the patient.

Diagnostic tests can be performed through either blood smear microscopy or PCR testing. PCR testing is more sensitive and specific than microscopy, bet the results are not available as fast as microscopy. 

This dataset contains images of blood cell slides with and without parasitic infection. The goal of this model is to help make the microscopic malarial testing even faster by attempting to remove some of the human element from the testing. We hope that multiple slides from a patient can be fed to this CNN model, which will quickly give out a result for whether or not the patient is infected and needs treatment.

Sources:

https://www.youtube.com/embed/TnzcwTyr6cE

https://www.cdc.gov/malaria/diagnosis_treatment/clinicians1.html#eval

We have been provided with images of infected blood cells, as well as uninfected blood cells. Our goal is to create a model that can differentiate between the two. Let's visualize the two side-by-side:

In [ ]:
uninfected_cell = '../input/files1/Malaria Cells/training_set/Uninfected/C100P61ThinF_IMG_20150918_144104_cell_128.png'
para_cell = '../input/files1/Malaria Cells/training_set/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_162.png'

plt.figure(1, figsize = (5 , 3))
plt.subplot(1 , 2 , 1)
plt.imshow(imread(uninfected_cell))
plt.title('Uninfected Cell')
plt.xticks([]) , plt.yticks([])

plt.subplot(1 , 2 , 2)
plt.imshow(imread(para_cell))
plt.title('Parasitized Cell')
plt.xticks([]) , plt.yticks([])

plt.show()

# Data Preprocessing
In order to fit a CNN model we need to make sure the dimensions (width & height) of each image are the same. Below, we will check the dimensions of all images, and then choose a final dimension to apply to all images prior to training the model. Having the same input picture size is important, as this determines the architecture of our CNN model. From the joint distribution plot below, we see that most pictures have height and width of around 130 pixels, so this is the reshaping size we select.

In [ ]:
len(os.listdir(../input/files1/Malaria Cells/training_set/Parasitized))

In [ ]:
#check dimensions
dim1 = []
dim2 = []

for image_filename in os.listdir("/kaggle/input/files1/Malaria Cells/testing_set/Parasitized"):

    img = Image.open("/kaggle/input/files1/Malaria Cells/testing_set/Parasitized/"+image_filename)
    d1, d2 = img.size
    dim1.append(d1)
    dim2.append(d2)

In [ ]:
#we have some small images, some large ones
sns.jointplot(x=dim1,y=dim2, kind='hist')

In [ ]:
#Resize to
image_shape = (130,130,3)

<h3>Adding Augmented Images</h3>
CNN models need a lot of training images to be able to create robust models. Instead of collecting more blood cells from people, we can create augmented data points using existing images by applying transformations on them, such as rotation or resizing them. Below you can see an original picture next to a slightly modified one.

In [ ]:
#To increase data points augment existing data
#Create more blood cell images with resize, rotation etc
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator(rotation_range=20,#randomyl rotate 0-20 degrees
                               width_shift_range = 0.10,#0.10: randomly change size using 0-10%
                               height_shift_range=0.10,#0.10: randomly change size using 0-10%
                               shear_range=0.10,
                               zoom_range=0.10,
                               horizontal_flip=True,
                               rescale=1/255,
                               fill_mode='nearest')


In [ ]:
para_img = imread(para_cell)

plt.figure(1, figsize = (5 , 3))
plt.subplot(1 , 2 , 1)
plt.imshow(para_img)
plt.title('Original Image')
plt.xticks([]) , plt.yticks([])

plt.subplot(1 , 2 , 2)
plt.imshow(image_gen.random_transform(para_img))
plt.title('Augmented Image')
plt.xticks([]) , plt.yticks([])

plt.show()

In [ ]:
#apply transformation to data set to artificially expand dataset
image_gen.flow_from_directory(train_path)

# Modeling

CNN models are great at image recognition, and widely used for this task. CNN's "look over" an image using a filter, usually 3x3 pixels. This helps them perform better at image recognition since they are looking at a region of NxN pixels, vs one pixel, so they can pick up on correlations in images. In contrast, if we were to fit a regular neural net model to an image, it would skip the feature extraction process, and just save the image inputs as a flattened layer using their pixel values as is. This would lead to loss of nearby pixel information, as well as make the input layer much larger.


![CNN Architecture](https://raw.githubusercontent.com/MnNqB/Malaria_CNN/main/CNN%20Architecture_MNB.PNG)

Our model has three convolutional layers, each using rectified linear activation units (ReLU). The convolutional layer applies different filters to try and detect patterns, and outputs a feature map (dot product). The ReLU activation returns only positive values while setting any negative dot products to zero. ReLU activations are widely used because they are more computationally efficient and show better convergence performance than other activation functions, such as the sigmoid function. Each convolutional layer's output is fed into a pooling layer. We are using a Max Pooling layer, which only keeps the maximum output from each quarter of the feature map. This reduced output is fed into the next layer.

To demonstrate a simple image and filter convolution, see CNN hand calculation below. A random filter is selected and applied using a dot product to the picture. A bias term is added, and the final value is added to the feature map. The process is started over after making the appropriate stride. In our model the stride is set to the default of 1.

![CNN hand calc](https://raw.githubusercontent.com/MnNqB/Malaria_CNN/main/CNN%20Example1.gif)

This makes it easy to see how a information from a large picture can be compressed down. We went from 36 values down to just 4 values. All while retaining information about the adjacent pixels. Once the image and filter dot product is calculated, a bias term is added. Once this is done across the whole image we have our feature map. Then we apply ReLU activation, which sets any negative numbers to zero. Finally, we apply max pooling, and put all our CNN output into one column (flattening). This is what gets fed into the Dense (neural net) layer. 

![CNN Hand Calc2](https://raw.githubusercontent.com/MnNqB/Malaria_CNN/main/CNN%20Example2.gif)


For more information please check out 'StatQuests' video on CNNs: 
https://www.youtube.com/watch?v=HGwBXDKFk9I&t=599s


In [ ]:
#Creating the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),input_shape=image_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

model.add(Dense(128,activation='relu'))

# randomly turn off 50% of neurons to reduce overfitting by randomly turning neurons off during training.
model.add(Dropout(0.5))

# binary: use sigmoid
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=2e-5),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)

#training batch size
batch_size = 20

In [ ]:
model.summary()

In [ ]:
#generate additional images
#image_shape[:2] because we only want dim1,dim2, not c from shape
train_image_gen = image_gen.flow_from_directory(train_path,target_size=image_shape[:2],
                                                color_mode='rgb',batch_size=batch_size,
                                                class_mode='binary')

In [ ]:
#generators
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

# Training

In [ ]:
results = model.fit(train_image_gen,epochs=20,
                              validation_data=test_image_gen,
                              callbacks=[early_stop])

# Model Evaluation

Our simple CNN model achieved 92% accuracy on the test images, which is great!
If we look at our model's confusion matrix, we find we achieved a recall score of 90% for detecting a parasitized cell (0).


In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['loss','val_loss']].plot()

In [ ]:
losses[['accuracy','val_accuracy']].plot()

In [ ]:
model.evaluate_generator(test_image_gen)

In [ ]:
pred_probabilities = model.predict(test_image_gen)

In [ ]:
predictions = pred_probabilities > 0.5

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
confusion_matrix(test_image_gen.classes,predictions)

# Misclassified Images
Next, let's look at some of the misclassified images to see what threw the model off. If we look at the first 6 image panel, it's easy to see how most of these pictures don't show an obvious sign of the parasite. Only images ‘c’ and ‘d’ look suspect.

The next 6 image panel shows uninfected cells, which got labelled as infected. All apart from image ‘i’ do appear to be normal cells. But the fact that some images are misclassified shows how our model could have performed even better with more accurate data.


In [ ]:
#Parasitized cells incorrectly classified examples
from tensorflow.keras.preprocessing import image

file_list = os.listdir("/kaggle/input/files1/Malaria Cells/testing_set/Parasitized")
file_index = 0
file_list_len = len(file_list)
mismatch_not_found = 1
n_found = 0
n_to_find = 6
idxs = np.zeros(n_to_find)
idxs = idxs.astype('uint32')
misclassified=[]
while n_found < n_to_find:

    while file_index < file_list_len and mismatch_not_found:

        image_filename = file_list[file_index]
        img = ("/kaggle/input/files1/Malaria Cells/testing_set/Parasitized/"+image_filename)

        my_image = image.load_img(img,target_size=image_shape)
        my_image_para = image.img_to_array(my_image)
        my_image_para = np.expand_dims(my_image_para, axis=0)
        predictions = model.predict(my_image_para)
        model_prediction = predictions.item(0)


        if (model_prediction != 0):
            mismatch_not_found = 0
            misclassified.append(file_index)
        else:
            file_index += 1    

    print(file_index)
    idxs[n_found] = file_index
    #my_image
    file_index += 1
    n_found +=1
    mismatch_not_found = 1

In [ ]:
image_filename = file_list[idxs[0]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Parasitized/"+image_filename)
my_image1 = image.load_img(img,target_size=image_shape)

image_filename = file_list[idxs[1]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Parasitized/"+image_filename)
my_image2 = image.load_img(img,target_size=image_shape)

image_filename = file_list[idxs[2]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Parasitized/"+image_filename)
my_image3 = image.load_img(img,target_size=image_shape)

image_filename = file_list[idxs[3]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Parasitized/"+image_filename)
my_image4 = image.load_img(img,target_size=image_shape)

image_filename = file_list[idxs[4]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Parasitized/"+image_filename)
my_image5 = image.load_img(img,target_size=image_shape)

image_filename = file_list[idxs[5]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Parasitized/"+image_filename)
my_image6 = image.load_img(img,target_size=image_shape)

In [ ]:
plt.figure(1, figsize = (15 , 7))
plt.subplot(2 , 3 , 1)
plt.imshow(my_image1)
plt.title('a')
plt.xticks([]) , plt.yticks([])

plt.subplot(2 , 3 , 2)
plt.imshow(my_image2)
plt.title('b')
plt.xticks([]) , plt.yticks([])

plt.subplot(2, 3 , 3)
plt.imshow(my_image3)
plt.title('c')
plt.xticks([]) , plt.yticks([])

plt.subplot(2 , 3 , 4)
plt.imshow(my_image4)
plt.title('d')
plt.xticks([]) , plt.yticks([])

plt.subplot(2 , 3 , 5)
plt.imshow(my_image5)
plt.title('e')
plt.xticks([]) , plt.yticks([])

plt.subplot(2, 3 , 6)
plt.imshow(my_image6)
plt.title('f')
plt.xticks([]) , plt.yticks([])

plt.show()

In [ ]:
#Uninfected cells incorrectly classified examples
from tensorflow.keras.preprocessing import image

file_list = os.listdir("/kaggle/input/files1/Malaria Cells/testing_set/Uninfected")
file_index = 0
file_list_len = len(file_list)
mismatch_not_found = 1
n_found = 0
n_to_find = 6
idxs_u = np.zeros(n_to_find)
idxs_u = idxs.astype('uint32')
misclassified=[]
while n_found < n_to_find:

    while file_index < file_list_len and mismatch_not_found:

        image_filename = file_list[file_index]
        img = ("/kaggle/input/files1/Malaria Cells/testing_set/Uninfected/"+image_filename)

        my_image = image.load_img(img,target_size=image_shape)
        my_image_para = image.img_to_array(my_image)
        my_image_para = np.expand_dims(my_image_para, axis=0)
        predictions = model.predict(my_image_para)
        model_prediction = predictions.item(0)


        if (model_prediction != 1):
            mismatch_not_found = 0
            misclassified.append(file_index)
        else:
            file_index += 1    

    print(file_index)
    idxs_u[n_found] = file_index
    #my_image
    file_index += 1
    n_found +=1
    mismatch_not_found = 1

In [ ]:
image_filename = file_list[idxs_u[0]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Uninfected/"+image_filename)
my_image7 = image.load_img(img,target_size=image_shape)

image_filename = file_list[idxs_u[1]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Uninfected/"+image_filename)
my_image8 = image.load_img(img,target_size=image_shape)

image_filename = file_list[idxs_u[2]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Uninfected/"+image_filename)
my_image9 = image.load_img(img,target_size=image_shape)

image_filename = file_list[idxs_u[3]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Uninfected/"+image_filename)
my_image10 = image.load_img(img,target_size=image_shape)

image_filename = file_list[idxs_u[4]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Uninfected/"+image_filename)
my_image11 = image.load_img(img,target_size=image_shape)

image_filename = file_list[idxs_u[5]]
img = ("/kaggle/input/files1/Malaria Cells/testing_set/Uninfected/"+image_filename)
my_image12 = image.load_img(img,target_size=image_shape)

In [ ]:
plt.figure(1, figsize = (15 , 7))
plt.subplot(2 , 3 , 1)
plt.imshow(my_image7)
plt.title('g')
plt.xticks([]) , plt.yticks([])

plt.subplot(2 , 3 , 2)
plt.imshow(my_image8)
plt.title('h')
plt.xticks([]) , plt.yticks([])

plt.subplot(2, 3 , 3)
plt.imshow(my_image9)
plt.title('i')
plt.xticks([]) , plt.yticks([])

plt.subplot(2 , 3 , 4)
plt.imshow(my_image10)
plt.title('j')
plt.xticks([]) , plt.yticks([])

plt.subplot(2 , 3 , 5)
plt.imshow(my_image11)
plt.title('k')
plt.xticks([]) , plt.yticks([])

plt.subplot(2, 3 , 6)
plt.imshow(my_image12)
plt.title('l')
plt.xticks([]) , plt.yticks([])
 
plt.show()

# Predicting on held-out images

Finally, we ran our model on the held-out images, and our model correctly classified both images. First is a parasitized cell and was correctly identified as 0 (parasitized). Second is an uninfected cell, which was also correctly classified as 1 (uninfected).

In [ ]:
test_image_gen.class_indices

In [ ]:
new_image1 = '../input/files1/Malaria Cells/single_prediction/Parasitised.png'
from tensorflow.keras.preprocessing import image
my_image_para = image.load_img(new_image1,target_size=image_shape)
my_image_para

In [ ]:
my_image_para = image.img_to_array(my_image_para)
my_image_para = np.expand_dims(my_image_para, axis=0)
model.predict(my_image_para)

In [ ]:
new_image2 = '../input/files1/Malaria Cells/single_prediction/Uninfected.png'
from tensorflow.keras.preprocessing import image
my_image_uninfected = image.load_img(new_image2,target_size=image_shape)
my_image_uninfected

In [ ]:
my_image_uninfected = image.img_to_array(my_image_uninfected)
my_image_uninfected = np.expand_dims(my_image_uninfected, axis=0)
m=model.predict(my_image_uninfected)

In [ ]:
from tensorflow.keras.models import load_model
model.save('./malaria_cnn.h5')